<a href="https://colab.research.google.com/github/nekoniii3/stable-diffusion-webui-colab/blob/main/XL1.0/SD_WebUI_CIVITAI%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CIVITAI用SDWebUI起動Colabノート**

## https://civitai.com/models

### **にあるモデルを試しに使うことができます。<br>モデルファイルのページのURLを入力して①②を実行してください。**
### **※現在はモデルファイル(Checkpoint)のみ対応でLORAは非対応**

**※Colab無料版で画像生成AIの利用が禁止となりました。有料版で実行してください。（警告が表示されます）**

### **⓪最初にランタイムのタイプをGPUに変更して下さい**

### **①SDWebUIリポジトリとモデルファイル取得**

### **↓ここにCivitaiのページのURLをセットして再生マーク実行**

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

# 変数定義
DOWN_PATH = "https://civitai.com/api/download/models/" #ダウンロードまでのパス

# URLを画面から取得
civitai_url = "https://civitai.com/models/105573/dream2reality" #@param {type:"string"}

# HTMLから情報取得
response = requests.get(civitai_url)
soup = BeautifulSoup(response.text,'html.parser')

# application/json情報取得
ap_json = soup.find(type="application/json")

# JSONをdictに
json_dict = json.loads(ap_json.text)

# checkpointType取得
checkpointType = json_dict["props"]["pageProps"]["trpcState"]["json"]["queries"][1]["state"]["data"]["type"]

# もしLoraなどcheckpointでなければ終了
if checkpointType != "Checkpoint":
  print("Checkpointモデルファイルではありません。（LORAなどではなくモデルファイル本体を指定して下さい。）")

else:

  # aria2インストール（ダウンロードを高速にするやつ）
  !apt -y install -qq aria2

  # モデルのIDを取得
  model_id = json_dict["props"]["pageProps"]["trpcState"]["json"]["queries"][0]["state"]["data"]["pages"][0]["items"][0]["modelVersionId"]

  # モデルのIDからダウンロードパス設定
  file_path = DOWN_PATH + str(model_id)

  # モデル名取得（取得できなくても続行）
  try:
    file_name = json_dict["props"]["pageProps"]["trpcState"]["json"]["queries"][1]["state"]["data"]["modelVersions"][0]["files"][0]["name"]
    file_name = file_name.replace(' ', '')
  except Exception as e:
    file_name = str(model_id) + ".safetensors"

  # stable-diffusion-webui（AUTOMATIC1111版）取得
  !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

  # dreamshaperXLモデルファイル取得
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M $file_path -d /content/stable-diffusion-webui/models/Stable-diffusion -o $file_name


### **②WebUI起動**

In [ ]:
%cd /content/stable-diffusion-webui
!python launch.py --share --xformers --enable-insecure-extension-access

### **生成画像をZIP化（左の[ファイル]直下にZIPファイルができます）**

In [ ]:
!zip -r /content/all_images.zip /content/stable-diffusion-webui/outputs

**[SAVE]した画像のみ**

In [ ]:
!zip -r /content/save_images.zip /content/stable-diffusion-webui/log/images